In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers.models import Pooling

pooling_layer = Pooling(768)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "../../data/models/roberta-finetuned-phemernr2/checkpoint-5410/",
    output_hidden_states=True, num_labels=4)

model.to("cuda")
print("Loading done")

Loading done


In [2]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs.to("cuda")
    token_logits = model(inputs['input_ids'])

    embeddings = token_logits.hidden_states[-1]

    inputs = {
        'attention_mask': inputs['attention_mask'],
        'token_embeddings': embeddings,
        'cls_token_embeddings': None
    }
    result = pooling_layer.forward(inputs)
    result = result['sentence_embedding'].cpu().detach().numpy()

    return result

def encode_and_save(texts, savefile = None):
    if not savefile:
        print("Please define savefile")
        return
    
    vectors = []
    for text in texts:
        vectors.append(encode_text(text))
    
    vectors = np.array(vectors).reshape((len(texts), 768))
    
    with open(f"../../data/vectors/{savefile}.txt", "wb") as f:
        np.savetxt(f, vectors, delimiter=',', fmt='%s')

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/phemernr2_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt,tvt2
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test,testting
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test,validation
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test,training
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training,training


In [4]:
data.shape

(6425, 9)

In [5]:
savefile = "Phemernr2_RoBERTa_base_finetuned_vectors"
texts = data['tweet_text'].tolist()

encode_and_save(texts, savefile)

In [6]:
vectors = np.loadtxt(f"../../data/vectors/{savefile}.txt", delimiter=",")
vectors.shape

(6425, 768)